In [1]:
import ee
import pandas as pd
from dask.distributed import Client, LocalCluster

# Initialize EE once on client
ee.Authenticate()
ee.Initialize(project='ee-spencerscw-snow')
SENTINEL = ee.ImageCollection("COPERNICUS/S1_GRD")

# Load depth_df once on client
depth_df = pd.read_json('depth.json')  # Adjust to your data source



In [2]:
def create_png(image, point, name, date_str):
    """Worker task - no depth_df reference"""

    # Actual image processing
    with managed_figure() as ax:
        ax.set_axis_off()
        ax.set_box_aspect(1)
        for i, band in enumerate(image.bandNames().getInfo()[:-1]):
            ee_image.geetools.plot(
                bands = [band],
                ax = ax,
                region = point,
                crs = "EPSG:3857",
                scale = 10,
                cmap = ("Greens", "Blues")[i],
                color = "k"
            )
            fig.savefig(f'temp{i}_{name}_{date_str}.png', bbox_inches="tight", pad_inches=0)

    with Image.open(f'temp0_{name}_{date_str}.png') as im1, Image.open(f'temp1_{name}_{date_str}.png') as im2:
        blended = Image.blend(im1, im2, alpha=.5)
        blended.save(f"images_snow_only/{name}_{date_str}.png")
        blended.close()

    for i in range(2):
        os.remove(f'temp{i}_{name}_{date_str}.png')


In [12]:
def generate_tasks():
    """Master process generates tasks using depth_df"""
    station_info = pd.read_json('./stations.json')

    for _, row in station_info.iterrows():
        name = row['stationTriplet']
        print(name)
        point_coords = [row.longitude, row.latitude]

        collection = ee.ImageCollection(SENTINEL) \
            .filterBounds(ee.Geometry.Point(point_coords).buffer(1000)) \
            .filterDate('2022-01-01', '2024-12-31')

        # Get image dates on client side
        dates = collection.aggregate_array('system:time_start') \
                         .map(lambda d: ee.Date(d).format('YYYY-MM-dd')) \
                         .getInfo()

        # Filter using client-side depth_df
        valid_dates = [
            date for date in dates 
            if not depth_df[(depth_df.name == name) & (depth_df.date == date)].empty
        ]
        print(valid_dates)

        # Get image IDs for valid dates
        image_ids = collection.filter(ee.Filter.inList('system:time_start', [
            ee.Date(date).millis().getInfo() for date in valid_dates
        ])).aggregate_array('system:index').getInfo()

        # Yield tasks for workers
        for image_id, date_str in zip(image_ids, valid_dates):
            yield (image_id, point_coords, name, date_str)



In [19]:
#station_info = pd.read_json('stations.json')
#stations = ee.FeatureCollection([
#    ee.Feature(
#        ee.Geometry.Point(row.longitude, row.latitude).buffer(1000),
#        {'name': row.stationTriplet}
#    ) for _, row in station_info.iterrows()
#])

depth_fc = ee.FeatureCollection([
    ee.Feature(
        ee.Geometry.Point(row.longitude, row.latitude).buffer(1000),
        {
            'name': row.name,
            'date': ee.Date(row.date).format('YYYY-MM-dd')
        }) for _, row in depth_df.iterrows()
])

sentinel_fc = SENTINEL.map(lambda img: ee.Feature(
    img.geometry(),
    {
        'date': img.date().format('YYYY-MM-dd'),
        'image': img
    }
))

join = ee.Join.saveAll(matchesKey='matches')
filter = ee.Filter.And(
    ee.Filter.equals(leftField='date', rightField='date'),
    ee.Filter.intersects(leftField='.geo', rightField='.geo')
)

joined = join.apply(
    primary = sentinel_fc,
    secondary = depth_fc,
    condition = filter
)

filtered_images = joined.map(lambda f: ee.Image(f.get('image')))
filtered_images

In [27]:
features = filtered_images.getInfo()['features']

EEException: Request payload size exceeds the limit: 10485760 bytes.

In [ ]:
features[0]

In [13]:
for task in generate_tasks():
    print(task)
    

ABY:CA:SNOW


[]
1F01A:BC:SNOW


[]
13E19:ID:SNOW


[]


15B07:ID:SNOW


...

['2022-01-03', '2022-01-15', '2022-01-27', '2022-02-08', '2022-02-20', '2022-03-04', '2022-03-16', '2022-03-28', '2022-04-09', '2022-04-21', '2022-05-03', '2022-05-15', '2022-05-27', '2022-10-30', '2022-11-11', '2022-11-23', '2022-12-05', '2022-12-17', '2022-12-29', '2023-01-10', '2023-01-22', '2023-02-03', '2023-02-15', '2023-02-27', '2023-03-11', '2023-03-23', '2023-04-04', '2023-04-16', '2023-04-28', '2023-05-10', '2023-05-22', '2023-06-03', '2023-06-15', '2023-06-27', '2023-07-09', '2023-11-18', '2023-11-30', '2023-12-12', '2023-12-24', '2024-01-05', '2024-01-17', '2024-01-29', '2024-02-10', '2024-02-22', '2024-03-05', '2024-03-17', '2024-03-29', '2024-04-10', '2024-04-22', '2024-05-04', '2024-05-28', '2024-06-09', '2024-10-19', '2024-10-31', '2024-11-12', '2024-11-24', '2024-12-06', '2024-12-18', '2024-12-30']


BFT:CA:SNOW


[]
931:WY:SNTL


['2022-01-05', '2022-01-17', '2022-01-29', '2022-03-06', '2022-03-18', '2022-03-30', '2022-04-11', '2022-04-23', '2022-05-05', '2022-05-17', '2022-05-29', '2022-06-10', '2022-11-01', '2022-11-13', '2022-11-25', '2022-12-07', '2022-12-19', '2022-12-31', '2023-01-12', '2023-01-24', '2023-02-05', '2023-02-17', '2023-03-01', '2023-03-13', '2023-03-25', '2023-04-06', '2023-04-18', '2023-04-30', '2023-05-12', '2023-10-15', '2023-10-27', '2023-11-08', '2023-11-20', '2023-12-02', '2023-12-14', '2023-12-26', '2024-01-07', '2024-01-19', '2024-01-31', '2024-02-12', '2024-02-24', '2024-03-07', '2024-03-19', '2024-03-31', '2024-04-12', '2024-04-24', '2024-05-06', '2024-05-18', '2024-05-30', '2024-10-21', '2024-11-02', '2024-11-14', '2024-11-26', '2024-12-08', '2024-12-20']


KeyboardInterrupt: 

In [0]:
# Configure cluster
cluster = LocalCluster(n_workers=32, threads_per_worker=1, memory_limit="4GB")
client = Client(cluster)

# Submit filtered tasks in batches
batch_size = 100  # Adjust based on memory
futures = []
for task in generate_tasks():
    future = client.submit(create_png, *task)
    futures.append(future)
    
    # Process in batches to avoid memory overload
    if len(futures) >= batch_size:
        client.gather(futures)
        futures = []

# Cleanup remaining tasks
if futures:
    client.gather(futures)

client.close()